In [ ]:
import pandas as pd
import sqlite3
# from datadings.tools import yield_threaded
from tqdm import tqdm

from models import Harmony

from models.clip import CLIP_VITL16
from models.dinov2 import vit_large
import argparse

import utils

In [ ]:
args = argparse.Namespace(arch='vit_small', batch_size_per_gpu=1, clip_grad=3.0, data='CC3M:/mnt/d/data/CC3M/cc3m/', dist_url='env://', drop_path_rate=0.1, epochs=100, freeze_last_layer=1, global_crops_scale=(0.4, 1.0), gpu=0, local_crops_number=8, local_crops_scale=(0.05, 0.4), local_rank=0, lr=0.0005, min_lr=1e-06, momentum_teacher=0.996, norm_last_layer=True, num_workers=10, objective='dino', optimizer='adamw', out_dim=65536, output_dir='/mnt/c/Users/Moham/Desktop/KAUST/results', patch_size=16, rank=0, saveckp_freq=20, seed=0, teacher_temp=0.04, use_bn_in_head=False, use_fp16=True, warmup_epochs=10, warmup_teacher_temp=0.04, warmup_teacher_temp_epochs=0, weight_decay=0.04, weight_decay_end=0.4, world_size=1)

In [ ]:
utils.init_distributed_mode(args)
utils.fix_random_seeds(args.seed)

model = Harmony(args=args)

In [ ]:
import torch
import torch.nn as nn

from timm.models.vision_transformer import PatchEmbed, Block
from vision_transformer import Block

from utils import get_2d_sincos_pos_embed

class GenerativePath(nn.Module):
    def __init__(self, image_encoder, patch_size=16, in_chans=3,
                embed_dim=1024, decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
                mlp_ratio=4, norm_layer=nn.LayerNorm, norm_pix_loss=False):
        super().__init__()

        self.norm_pix_loss = norm_pix_loss
        self.image_encoder = image_encoder
        self.patch_embed = image_encoder.patch_embed
        num_patches = self.image_encoder.patch_embed.num_patches

        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)
        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))
        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, qk_scale=None, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, patch_size**2 * in_chans, bias=True) # decoder to patch

    def initialize_deocder_weights(self):
        
        decoder_pos_embed = get_2d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        torch.nn.init.normal_(self.mask_token, std=.02)

    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 *3)
        """
        print(self.patch_embed.patch_size)
        p = self.patch_embed.patch_size
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], 3, h, p, w, p))
        x = torch.einsum('nchpwq->nhwpqc', x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * 3))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size
        h = w = int(x.shape[1]**.5)
        assert h * w == x.shape[1]
        
        x = x.reshape(shape=(x.shape[0], h, w, p, p, 3))
        x = torch.einsum('nhwpqc->nchpwq', x)
        imgs = x.reshape(shape=(x.shape[0], 3, h * p, h * p))
        return imgs

    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward_encoder(self, x, mask_ratio):
        # embed patches
        x = self.image_encoder.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.image_encoder.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)

        # append cls token
        cls_token = self.image_encoder.cls_token + self.image_encoder.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.image_encoder.blocks:
            x = blk(x)
        x = self.image_encoder.norm(x)

        return x, mask, ids_restore

    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.e-6)**.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss

    def forward(self, imgs, mask_ratio=0.75):
        latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
        loss = self.forward_loss(imgs, pred, mask)
        return loss, pred, mask

In [ ]:
s = GenerativePath(image_encoder=model.image_encoder, embed_dim=384).cuda()
r = torch.rand([1, 3, 224, 224]).cuda()

In [ ]:
l, p, m = s(r)

In [ ]:
clip = CLIP_VITL16()

In [ ]:
import torch

In [ ]:
import sys
import os

In [ ]:
a = [f for f in os.scandir("/mnt/d/data/CC3M/cc3m/") if f.is_dir()]

In [ ]:
def save_images_in_folders(folders):
    images_paths = []
    for folder in folders:
        for filename in os.listdir(folder):
            if os.path.isfile(os.path.join(folder, filename)):

In [ ]:
paths = save_images_in_folders(a)

In [ ]:
from PIL import Image

def save_image_captions_in_folders(folders, root):
    images_paths  = []
    captions_path = []
    for folder in folders:
        for filename in os.listdir(folder):
            if os.path.isfile(os.path.join(folder, filename)):
                if ".jpg" in filename or ".png" in filename:
                    images_paths.append(root + os.sep + folder.name + os.sep + filename)
                elif ".txt" in filename:
                    captions_path.append(root + os.sep + folder.name + os.sep + filename)
        break
    return images_paths, captions_path

class CC3M(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root
        self.folders =  [f for f in os.scandir(root) if f.is_dir()]
        self.images, self.captions = save_image_captions_in_folders(self.folders, self.root)

        assert len(self.captions) == len(self.images)
        print("Number of images loaded in CC3M are: ", {self.__len__()})

    def __len__(self):
        return len(self.images)

    def get_image_caption_pair(self, idx):
        image = Image.open(self.images[idx])
        
        caption_file = open(self.captions[idx])
        caption = caption_file.read()
        caption_file.close()

        return image, caption


    def __getitem__(self, idx):
        image, caption = self.get_image_caption_pair(idx)
        return image, caption

In [ ]:
a = CC3M(root="/mnt/d/data/CC3M/cc3m")

In [ ]:
i, t = a[0]

In [ ]:
import tarfile

# Open the tar file
tar = tarfile.open(r'D:\data\CC3M\cc3m\00000.tar')

# Loop over each member
for member in tar.getmembers():
    print(member)
    # Extract each file as a file object
    f = tar.extractfile(member)
    if f is not None:
        # Read the contents
        content = f.read()

# Close the tar file
tar.close()
